In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve
from sklearn.metrics import classification_report

import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
!pwd

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
import pandas_profiling
train.profile_report()

In [ ]:
data = pd.concat([train, test], sort = False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.hist(train.loc[train['Survived'] ==0, 'Age'].dropna(), bins=30, alpha = 0.5, label='0')
plt.hist(train.loc[train['Survived'] ==1, 'Age'].dropna(), bins=30, alpha = 0.5, label='1')
plt.xlabel('Age')
plt.ylabel('count')
plt.legend(title='Survived')

In [ ]:
sns.countplot(x='SibSp', hue='Survived', data = train)
plt.legend(loc='upper right', title='Survived')

In [ ]:
sns.countplot(x='Parch', hue='Survived', data = train)
plt.legend(loc='upper right', title='Survived')

In [ ]:
# The higher the fare, the higher the survival rate.
plt.hist(train.loc[train['Survived'] ==0, 'Fare'].dropna(), range=(0, 250), bins=30, alpha = 0.5, label='0')
plt.hist(train.loc[train['Survived'] ==1, 'Fare'].dropna(), range=(0, 250), bins=30, alpha = 0.5, label='1')
plt.xlabel('Fare')
plt.ylabel('count')
plt.legend(title='Survived')
plt.xlim(-5, 250)

In [ ]:
sns.countplot(x='Pclass', hue='Survived', data = train)

In [ ]:
sns.countplot(x='Sex', hue='Survived', data = train)

In [ ]:
sns.countplot(x='Embarked', hue='Survived', data = train)

In [ ]:
data.info()


In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
#Embarked
data['Embarked'].value_counts() #data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

In [ ]:
 
#  3   Name         200000 non-null  object                          -- label_cols
#  4   Sex          200000 non-null  object                          -- label_cols
#  5   Age          193416 non-null  float64    -- missing value 
#  8   Ticket       190754 non-null  object     -- missing value     -- label_cols
#  9   Fare         199732 non-null  float64    -- missing value 
#  10  Cabin        64268 non-null   object     -- missing value     -- onehot_cols
#  11  Embarked     199500 non-null  object     -- missing value     -- onehot_cols

# https://www.kaggle.com/hiro5299834/tps-apr-2021-pseudo-labeling-voting-ensemble


# Name, take only surnames
data['Name'] = data['Name'].map(lambda x: x.split(',')[0])

age_avg = data['Age'].mean()
age_std = data['Age'].std()
data['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)


# Ticket, fillna with 'X', split string and take first split 
data['Ticket'] = data['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

# Cabin, fillna with 'X' and take first letter
data['Cabin'] = data['Cabin'].fillna('X').map(lambda x: x[0].strip())

# Embarked, fillna with 'X' value
data['Embarked'] = data['Embarked'].fillna('X')

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.Name.value_counts()

In [ ]:
label_cols = ['Name', 'Ticket', 'Sex']
onehot_cols = ['Cabin', 'Embarked']
numerical_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
data

In [ ]:
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

scaler = StandardScaler()

In [ ]:
onehot_encoded_df = pd.get_dummies(data[onehot_cols])
onehot_encoded_df.reset_index(inplace=True, drop=True)
onehot_encoded_df

In [ ]:
label_encoded_df = data[label_cols].apply(label_encoder)
label_encoded_df.reset_index(inplace=True, drop=True)
label_encoded_df

In [ ]:
numerical_df = pd.DataFrame(scaler.fit_transform(data[numerical_cols]), columns=numerical_cols)
numerical_df.reset_index(inplace=True, drop=True)
numerical_df

In [ ]:
target_df = data['Survived']
target_df.reset_index(inplace=True, drop=True)
target_df

In [ ]:
data = pd.concat([numerical_df, label_encoded_df, onehot_encoded_df, target_df], axis=1) 
data = data.reset_index(drop=True)
data

In [ ]:
train = data[:len(train)]
test = data[len(train):]
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
x_test = test.drop('Survived', axis=1)
y_test= test['Survived']

In [ ]:
data.isnull().sum()

In [ ]:
X_train.info()

In [ ]:
X_train.head()

In [ ]:
plt.subplots(figsize=(16,12))
sns.heatmap(data.corr(), annot=True, cmap="RdYlGn")
fig.tight_layout()
fig.show()

# Logistic Regression Model

In [ ]:
clf = LogisticRegression(penalty='l2', solver = 'sag', random_state=0) # 답을 찾는 방법.. 확률적 경사하강법 
clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(x_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
accuracy_score(y_test, predictions)  # 이걸로 안돼는뎅 ㅎㅎ

In [ ]:
probabilities = clf.predict_proba(x_test)[:, 1]
probabilities.shape

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, probabilities)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(fpr, tpr)
ax.set_xlabel("False Positive Rate (1 - Specificity)")
ax.set_ylabel("True Positive Rate (Sensitivity, Recall)")
ax.set_title("ROC Plot of Titanic Data");

In [ ]:
fpr, tpr, thresholds 

# Decision Tree Model

In [ ]:
N_ESTIMATORS = 1000
N_SPLITS = 10
SEED = 2021
EARLY_STOPPING_ROUNDS = 100
VERBOSE = 100

In [ ]:
# https://www.kaggle.com/hiro5299834/tps-apr-2021-pseudo-labeling-voting-ensemble 

# Tuning the DecisionTreeClassifier by the GridSearchCV
parameters = {
    'max_depth': np.arange(2, 5, dtype=int),
    'min_samples_leaf':  np.arange(2, 5, dtype=int)
}

classifier = DecisionTreeClassifier(random_state=2021)

model = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=10,
    n_jobs=-1)
model.fit(X_train, y_train)

best_parameters = model.best_params_
print(best_parameters)

# {'max_depth': 4, 'min_samples_leaf': 2}

In [ ]:
SEED = 2021
dtm_oof = np.zeros(train.shape[0])
dtm_preds = np.zeros(test.shape[0])
feature_importances = pd.DataFrame()

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for fold, (train_idx, valid_idx) in enumerate(skf.split(data,data['Survived'])):
    print(f"===== FOLD {fold} =====")
    oof_idx = np.array([idx for idx in valid_idx if idx < train.shape[0]])
    preds_idx = np.array([idx for idx in valid_idx if idx >= train.shape[0]])

    X_train, y_train = data.iloc[train_idx].drop('Survived', axis=1), data.iloc[train_idx]['Survived']
    X_valid, y_valid = data.iloc[oof_idx].drop('Survived', axis=1), data.iloc[oof_idx]['Survived']
    X_test = data.iloc[preds_idx].drop('Survived', axis=1)
    
    model = DecisionTreeClassifier(
        max_depth=best_parameters['max_depth'],
        min_samples_leaf=best_parameters['min_samples_leaf'],
        random_state=SEED
    )
    model.fit(X_train, y_train)
    
    dtm_oof[oof_idx] = model.predict(X_valid)
    dtm_preds[preds_idx-train.shape[0]] = model.predict(x_test)
    
    acc_score = accuracy_score(y_valid, np.where(dtm_oof[oof_idx]>0.5, 1, 0))
    print(f"===== ACCURACY SCORE {acc_score:.6f} =====\n")
    
acc_score = accuracy_score(data[:train.shape[0]]['Survived'], np.where(dtm_oof>0.5, 1, 0))
print(f"===== ACCURACY SCORE {acc_score:.6f} =====")

In [ ]:
# plot tree
dot_data = export_graphviz(
    model,
    out_file=None,
    feature_names=X_train.columns,
    class_names=['0', '1'],
    filled=True,
    rounded=False,
    special_characters=True,
    precision=3
)
graph = graphviz.Source(dot_data)
graph 

In [ ]:
LGB Boost
Random Forest
